<a href="https://colab.research.google.com/github/DenisseDB/MachineLearning-BankAnalyses/blob/main/BankSellClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import tensorflow as tf
import numpy as np
import pandas as pd
# para el one hot encoder
import joblib

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Accedemos tanto al modelo como al encoder para los inputs del usuario, de tal forma que puedan ser transfromados a binario

In [85]:
model = tf.keras.models.load_model("/content/drive/MyDrive/BankData/banksell.keras")
oneHotEncoder = joblib.load("/content/drive/MyDrive/BankData/onehotencoder.pkl")

Columnas necesarias
- needEncoder, son aquellas que necesitan pasar por el oneHotEncoder
- noEncoder, ya son numericas, por lo tanto, no es necesaria una transformacion

In [86]:
columns = ['age', 'marital', 'education', 'default', 'loan', 'duration']
needEncoder = ['marital', 'education', 'default', 'loan']
noEncoder = ['age', 'duration']

Si el input necesita encoder, lo guardamos tal cual se recibe, pero, si no necesita encoder, lo guardamos como un float



*   Age -> Edad -> Input numerico
*   Maritial -> Estado civil -> single, married, divorced (valido para widowed) o unknown
* Education -> Nivel de estudios -> primary, secondary, tertiary o unknown
* Default -> ¿Tiene mal historial con deudas no pagadas? -> Yes o No
* Loan -> ¿Tiene deudas pendientes? -> Yes o No
* Duration -> Duración de la llamada -> Minutoshas  



In [87]:
userInput = {}
print("Plese enter the costumer information:")

for column in columns:
  if column in needEncoder:
    userInput[column] = input(f"{column}: ")
  else:
    userInput[column] = float(input(f"{column}: "))

Plese enter the costumer information:
age: 23
marital: SINGLE
education: TERTIARY
default: NO
loan: NO
duration: 250


Creamos el dataframe con los datos del usuario que son los que se van a predecir con ayuda del modelo

In [88]:
df = pd.DataFrame(userInput, index=[0])


A las columnas con inputs que necesitan el One Hot Encoder, se lo aplicamos, para despues, ahora si, juntar ambas columnas y tener el dataframe completo

In [89]:
encoded = oneHotEncoder.transform(df[needEncoder])
encoded_df = pd.DataFrame(encoded, columns = oneHotEncoder.get_feature_names_out())
concat = pd.concat([encoded_df, df[noEncoder].reset_index(drop=True)], axis=1)

Le pasamos los datos al modelo para que nos de una predicción de acuerdo a los datos que le hemos mandado

In [90]:
pred = model.predict(concat.to_numpy())
prob = pred[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


In [91]:

print("El cliente SÍ aceptará la tarjeta" if prob >= 0.5 else "El cliente NO aceptará la tarjeta")

El cliente SÍ aceptará la tarjeta
